In [17]:
import pickle
import numpy as np
import pandas as pd
import organizeOU
from collections import Counter,defaultdict
import boxScoreHelper

In [36]:

def readGames(year = '2014'):
    # need to convert CHA to CHO
    with open('../Data/ou/allOU2U.pkl','rb') as handle:
        ouResults = pickle.load(handle)
    glres = ouResults[year]
    with open('../Data/Clean/TeamBoxByYear/results_{0}.pkl'.format(year),'rb') as handle:
            bbres = pickle.load(handle)
    
    return glres,bbres


In [111]:
def line2tuple(res):
    out = {}
    for game in res:
        h,a = game[-3:],organizeOU.full2abv[res[game][0]]
        out[game] = (h,a,res[game][1])
    return out
def box2tuple(res):
    out = {}
    fixCHO = lambda x:'CHA' if x=='CHO' else x
    for game in sorted(res):
        cgame = res[game]
        fscore = boxScoreHelper.getFinalScore(cgame)
        h,a = fixCHO(cgame["away"].upper()),fixCHO(cgame["home"].upper())
        out[game] = (h,a,fscore)
    return out
def bb2teamList(bb):
    games_ordered = []
    games_byteam = defaultdict(list)
    fixCHO = lambda x:'CHA' if x=='CHO' else x
    for game in sorted(bb):
        cgame = bb[game]
        fscore = boxScoreHelper.getFinalScore(cgame)
        h,a = fixCHO(cgame["away"].upper()),fixCHO(cgame["home"].upper())
        team = fixCHO(game[-3:])
        games_byteam[team].append((h,a,fscore,game))
    return games_byteam


def sortGames(year='2014',need_check = False,fixdict = {'BRK':'NJN','OKC':'SEA','NOP':'NOH','CHA':'CHO'}):
    ''' Return betting id to bbref id
    '''
    if year == '2006':
        fixdict = {'BRK':'NJN','OKC':'SEA','NOP':'NOK'}
    gl,bb = readGames(year)
    bbteams = bb2teamList(bb)
    '''
    - Read through sorted games
        -keep sorted list of home team
        - keep sorted list of teams games in dict
    - loop through sorted list of home teams
        - get games from sorted home list
    '''
    teamCounter = defaultdict(int)
    outdict = {}
    
    for game in sorted(gl):
        team = game[-3:]
        i = teamCounter[team]
        teamCounter[team] += 1
        try:
            
            bbid = bbteams[team][i]
        except IndexError:
            try:
                
                if team in fixdict:
                    team = fixdict[team]
                bbid = bbteams[team][i]
            except IndexError:
                print("Index Error:  ",team,year)
        outdict[game] = bbid 
    if need_check == False:
        return {game:outdict[game][-1] for game in outdict}
    
    gans = line2tuple(gl)
    bans = box2tuple(bb)    
    working = all( (gans[i][0]==bans[outdict[i][-1]][0] for i in outdict ))

    print("Games appear to be matched up: ",working)

    #dictionary of betting games to bbgames
    return {game:outdict[game][-1] for game in outdict}






# print("X: ",x)


In [112]:

for i in range(2006,2015):
    if i == 2006:
        sortGames(str(i))
    else:
        sortGames(str(i))